In [ ]:
# Future improvements
# Create JSON array to assign IDs, 
#     keep track of PDF files process (each step?) etc.
# Implement 'fi'/'fl' check at beginning and end of words

In [5]:
# Imports
import os # For file/directory interaction
import time, sys
import PyPDF2 # Library for converting PDF to text
from datetime import datetime, date # For log data
import decimal # Exceptions caught from this package
import re # For text replacement
import spacy # Pipeline processes (stopword and punctuation removal, lemmatization)
from nltk.stem.snowball import SnowballStemmer # Pipeline process for stemming
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
nlp = spacy.load("en_core_web_sm")
txtFilesDir = 'Text Files'
absolute = 'C:/Users/micah/Documents/IWU/CIS Practicum/Files'

In [28]:
# Pre-condition: All PDF files to be processed are in the sub-directory
#     pdfDir, and pdfDir is in absPath. absPath is by default the 
#     directory in which the program is executed
# Post-condition: All PDF files processed without error are converted to
#     text files which are placed in a new sub-directory 'Text Files'
def pdfToText(pdfDir, absPath = os.getcwd(), txtDir = txtFilesDir):
    pdfDirectory = absPath+'/'+pdfDir
    txtDirectory = absPath+'/'+txtDir
    if pdfDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
# Creates 'Text Files' directory for converted PDFs
    if txtDir not in os.listdir(absPath):
        os.mkdir(txtDirectory)
    
    docNum = 0
    stopAt = 150
    totalNum = len([file for file in os.scandir(pdfDirectory) if file.name.endswith('.pdf')])
    with open(absPath+'/'+'log.txt', 'a+', encoding="utf-8") as log:
        log.write("PDF to Text\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")        
# Moves on to next entity if the current entity is not a PDF
        for entity in os.scandir(pdfDirectory):
            if not entity.name.endswith('.pdf') or entity.name[0] in '1234567890':
                continue
            docNum += 1
            index = -4 # Remove '.pdf' from file name when creating '.txt' file
            fileName = entity.name[:index]+'.txt'
            print("Now on '"+entity.name+"'. . . ", end='')
            
# This block attempts to read the PDF file, extract text from each page,
#     and write the text to a text file with the same name
# Some documents are protected, corrupted, etc. and text cannot be extracted
# Exceptions are recorded in log.txt
# hasError remains true until each step in the try block is complete
            if fileName not in os.listdir(txtDirectory): 
                hasError = True
                with open(pdfDirectory+'/'+entity.name, 'rb') as pdfFileObject:
                    try:
                        pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
                        numPages = pdfReader.getNumPages()
                        pageObject = pdfReader.getPage(0)
                        textFile = open(txtDirectory+'/'+fileName, 'a+', encoding="utf-8")
                        i = 0;
                        while i < numPages:
                            pageObject = pdfReader.getPage(i)
                            textFile.write(pageObject.extractText())
                            i += 1
                        print("done\n")
                        hasError = False
#                     except TypeError as e:
#                         log.write("TypeError: " + entity.name + ":\n" + str(e))
#                     except PyPDF2.utils.PdfReadError as e:
#                         log.write("PdfReadError: " + entity.name + ":\n" + str(e))
#                     except OSError as e:
#                         log.write("OSError: " + entity.name + ":\n" + str(e))
#                     except decimal.InvalidOperation as e:
#                         log.write("InvalidOperation: " + entity.name + ":\n" + str(e))
                    except Exception as e:
                        log.write(str(docNum)+": " + entity.name + ": \n\t" + str(e)+"\n")
                        textFile.close()
                        if fileName in os.listdir(txtDirectory):
                            os.remove(txtDirectory+'/'+fileName)
                    pdfFileObject.close()

                if hasError:
                    print("there was an error reading this document. See log for details. Reference number "+str(docNum)+".\n")
            else:
                max = 10
                if len(fileName) > max:
                    print('"'+fileName[:11]+'"' + '...', end='')
                else:
                    print('"'+fileName+'"', end='')
                print(' already exists\n')
            if docNum == stopAt:
                print("PDF to Text was stopped after "+str(docNum)+" documents.")
                break
        log.write("\n\n")
pdfToText('PDF', absolute)

Now on 'A  cognitive affective model of organizational communication for designing IT.pdf'. . . "A  cognitiv"... already exists

Now on 'A Bible for the Disability Field.pdf'. . . "A Bible for"... already exists

Now on 'A Comparisoon of Binarization Methods for Historical Archive Documents.pdf'. . . "A Compariso"... already exists

Now on 'A light-weight text image processing method for handheld embedded cameras.pdf'. . . "A light-wei"... already exists

Now on 'A Mathematical Theory of Communication.pdf'. . . "A Mathemati"... already exists

Now on 'A Model for Types and Levels of Human Interaction with Automation.pdf'. . . "A Model for"... already exists

Now on 'A Parser for Real-Time Speech Synthesis of Conversational Texts.pdf'. . . "A Parser fo"... already exists

Now on 'A Physically Based Approach to 2-D Shape Blending.pdf'. . . there was an error reading this document. See log for details. Reference number 8.

Now on 'A Review of 326 Children with Developmental and Physical D

Now on 'Augmented Reality Platform for Mass Casualty Incidents.pdf'. . . done

Now on 'Aural Information Architectures.pdf'. . . done

Now on 'AutobiographicalDesign.pdf'. . . done

Now on 'Automatic Evaluation of Document Binarization Results.pdf'. . . done

Now on 'AutomaticPageAnalysis.pdf'. . . done

Now on 'Automation Induced Complacency.pdf'. . . done

Now on 'Avios-DSR-paper.pdf'. . . done

Now on 'awfechallengeoreilly.pdf'. . . done

Now on 'A_Guide_to_Clinical_Engineering.pdf'. . . done

Now on 'A_rational_design_process_How_and_why_to_fake_it.pdf'. . . done

Now on 'A_Survey_of_the_Hough_Transform.pdf'. . . done

Now on 'BachmanInventedDBMS.pdf'. . . done

Now on 'BadSoftwareArchitectue is a People Problem.pdf'. . . done

Now on 'Baltes_Boris_B.pdf'. . . done

Now on 'BDA.pdf'. . . done

Now on 'Bearings-only maneuvering target tracking based on truncated quadrature Kalman filtering.pdf'. . . done

Now on 'Bearings-Only Tracking Using Augmented Ensemble Kalman Filter.pdf'. . 

In [29]:
# Work on 10 (good) files at a time until pipeline works
#   then incrementally add files and clean up errors

# Function to remove \n
filesDir = 'Text Files'
def rmvN(filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    if filesDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    for entity in os.scandir(directory):
        with open(directory+'/'+entity.name, 'r+', encoding='utf-8') as f:
            text = f.read()
            text = re.sub('-\n', '', text)
            text = re.sub('\n', '', text)
            f.seek(0)
            f.write(text)
            f.truncate()
rmvN(filesDir, absolute)

In [30]:
# Funtion to move files without spaces to new 'Without Spaces' directory         
def checkSpaces(filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    if filesDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    wsDir = 'Without Spaces'
    if wsDir not in os.listdir(absPath):
        os.mkdir(absPath+'/'+wsDir)
        
    with open(absPath+'/'+'No Spaces.txt', 'a+', encoding='utf-8') as noSpaces: 
        noSpaces.write("Check Spaces\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")
        for entity in os.scandir(directory):
            f = open(directory+'/'+entity.name, 'r', encoding='utf-8')
            text = f.read()
            split = text.split(' ')
            if len(split) < len(text)/10 or text == '':
                f.close()
                noSpaces.write(entity.name+'\n')
                if entity.name not in os.listdir(absPath+'/'+wsDir):
                    os.rename(directory+'/'+entity.name, absPath+'/'+wsDir+'/'+entity.name)
                else:
                    os.remove(directory+'/'+entity.name)
        noSpaces.write('\n\n')
checkSpaces(filesDir, absolute) 

In [31]:
# Function to remove stopwords
# NLTK or SpaCy
# Inverted File: gram:[doc1, doc3] or gram:[[doc1,freq], [doc3,freq]]
def rmvStopWords(nlp, filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    if filesDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    stemmer = SnowballStemmer(language='english')
    for entity in os.scandir(absPath+'/'+filesDir):
        with open(directory+'/'+entity.name, 'r+', encoding='utf-8') as f:
            doc = nlp(f.read())
            noStopWords = [stemmer.stem(token.lemma_.lower()) for token in doc if not token.is_stop and not token.is_punct]
#             noStopWords = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
#             Quotation marks appear as 'fi' in text files (merged with words)
#             maybe compare to words that start with fi
            f.seek(0)
            f.write(" ".join(noStopWords))
            #                 f.write(token.orth_.lower() + ' ')
            f.truncate()

rmvStopWords(nlp, filesDir, absolute)

In [32]:
# Calculate cosine similarity score for each document
def tfdif_cossim(filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    if filesDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    vectorizer = TfidfVectorizer(ngram_range=(3,3), stop_words=None, use_idf=True)
    X2 = vectorizer.fit_transform([open(directory+'/'+file, "r", encoding='utf-8').read() for file in os.listdir(directory)])
#     X2 = vectorizer.fit_transform(open(directory+'/'+'A Bible for the Disability Field.txt', 'r', encoding='utf-8'))
    features = vectorizer.get_feature_names()
#     print(features)
    scores = (X2.toarray()) 
#     print("\n\nScores : \n", scores) 

    # Getting top ranking features 
    sums = X2.sum(axis = 0) 
    data1 = [] 
    for col, term in enumerate(features): 
        data1.append( (term, sums[0, col] )) 
    ranking = pd.DataFrame(data1, columns = ['term', 'rank']) 
    words = (ranking.sort_values('rank', ascending = False)) 
    print ("\n\nWords : \n", words.head(50)) 
    
tfdif_cossim(filesDir, absolute)



Words : 
                               term      rank
117746         cohn editormik cohn  1.303682
173640    editormik cohn editormik  0.977762
5136                      11 11 11  0.641984
397587            scrum scrum meet  0.496957
66811         agil softwar develop  0.494473
71798    ambient intellig everyday  0.481594
390607    roadmap ambient intellig  0.474334
185258        esto roadmap ambient  0.471914
70                        00 pm 00  0.467379
341521                    pm 00 pm  0.467379
83937      assist technolog outcom  0.460861
168510           docu my 20earticl  0.450071
302678       my 20earticl emmerson  0.450071
471344          usag center design  0.442381
148659        decis support system  0.428163
446562    technolog outcom benefit  0.421394
0                         00 00 00  0.415448
28166                  30 jan 2017  0.408248
8785                1412 6980v9 cs  0.408248
270094                   lg 30 jan  0.408248
44311                 6980v9 cs lg  0.40824

In [ ]:
# Trigram parser-based inverted file 
# (TF-DIF to remove trigrams common to most or all documents)

In [ ]:
# Clustering algorithm based on trigram inverted file

In [ ]:
# Add bigram parser-based info to inverted file

In [ ]:
# Implement clustering on bigram inverted file